In [1]:
# packages for dealing with Sidra API
import requests
import json

# packages for treating the data
import pandas as pd
import pandana as pdna
from pandana.loaders import osm
import numpy as np
import unidecode

# packages for dealing with geographical data
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import osmnx as ox
from osmnx.distance import get_nearest_node


To hide the warning that says "`should_run_async` will not call `transform_cell` automatically in the future", we use the code below while the [issue](https://github.com/ipython/ipykernel/issues/540) isn't solved.

In [2]:
import warnings
# warnings.filterwarnings("ignore")
# warnings.simplefilter('ignore')

## Building the dataset

This project aims to measure the pedestrian acessibility in Natal, Brazil. With that in mind, we use data from [Sidra API](http://api.sidra.ibge.gov.br/) regarding the neighborhoods, average income and population size. Further, we calculate the smallest distance from a central point of the neighborhood to an amenity. All this informations togheter will become our dataset.

In [3]:
# Recover income by neighborhood in Natal (2010)
# from Sidra's Table 3170 (http://api.sidra.ibge.gov.br/desctabapi.aspx?c=3170)
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

endnode = "http://api.sidra.ibge.gov.br/values/t/3170/p/2010/v/842/N102/in%20n6%202408102"
response = requests.get(endnode,headers=headers)
raw_data_income = response.json()

In [4]:
# Recover the number of inhabitants
endnode_people = "http://api.sidra.ibge.gov.br/values/t/3170/p/2010/v/841/N102/in%20n6%202408102"
response = requests.get(endnode_people,headers=headers)
raw_data_people = response.json()

The variables 842 and 841 give us, respectivelly, the information about income and population of each neighborhood. Now we take a look at the responses to find out the meaning of each key-value pair.

In [5]:
raw_data_income[0]

{'NC': 'Nível Territorial (Código)',
 'NN': 'Nível Territorial',
 'MC': 'Unidade de Medida (Código)',
 'MN': 'Unidade de Medida',
 'V': 'Valor',
 'D1C': 'Ano (Código)',
 'D1N': 'Ano',
 'D2C': 'Variável (Código)',
 'D2N': 'Variável',
 'D3C': 'Bairro (Código)',
 'D3N': 'Bairro',
 'D4C': 'Situação do domicílio (Código)',
 'D4N': 'Situação do domicílio',
 'D5C': 'Sexo (Código)',
 'D5N': 'Sexo',
 'D6C': 'Grupo de idade (Código)',
 'D6N': 'Grupo de idade'}

In [6]:
raw_data_people[0]

{'NC': 'Nível Territorial (Código)',
 'NN': 'Nível Territorial',
 'MC': 'Unidade de Medida (Código)',
 'MN': 'Unidade de Medida',
 'V': 'Valor',
 'D1C': 'Ano (Código)',
 'D1N': 'Ano',
 'D2C': 'Variável (Código)',
 'D2N': 'Variável',
 'D3C': 'Bairro (Código)',
 'D3N': 'Bairro',
 'D4C': 'Situação do domicílio (Código)',
 'D4N': 'Situação do domicílio',
 'D5C': 'Sexo (Código)',
 'D5N': 'Sexo',
 'D6C': 'Grupo de idade (Código)',
 'D6N': 'Grupo de idade'}

The keys for neighborhood code, neighborhood name, and value for each variable (income and population), are, in this order, `D3C`, `D3N` and `V`. We use these to build the first dataframes. Only the dataset for income will have the name, because the two datasets will be merged using the neighboorhood code.

In [7]:
# create a dataset for each json
income_df = pd.DataFrame(raw_data_income, columns=['D3C', 'D3N', 'V'])
income_df.drop(labels=0, inplace=True)

people_df = pd.DataFrame(raw_data_people, columns=['D3C', 'V'])
people_df.drop(labels=0, inplace=True)

In [8]:
# quick checkout on the results
print(income_df.head())
print(people_df.head())

          D3C                         D3N        V
1  2408102001    Santos Reis - Natal - RN   984.31
2  2408102002  Praia do Meio - Natal - RN  1658.16
3  2408102003          Rocas - Natal - RN   969.39
4  2408102004        Ribeira - Natal - RN  2825.82
5  2408102005     Petrópolis - Natal - RN  4736.63
          D3C     V
1  2408102001  2989
2  2408102002  2810
3  2408102003  5806
4  2408102004  1453
5  2408102005  3288


In [9]:
neighborhood_df = income_df.merge(people_df, on='D3C')
neighborhood_df.head()

,D3C,D3N,V_x,V_y
0,2408102001,Santos Reis - Natal - RN,984.31,2989
1,2408102002,Praia do Meio - Natal - RN,1658.16,2810
2,2408102003,Rocas - Natal - RN,969.39,5806
3,2408102004,Ribeira - Natal - RN,2825.82,1453
4,2408102005,Petrópolis - Natal - RN,4736.63,3288


Finally, the columns will be renamed. There is a need to remove any accents from the names of the neighborhoods. This is necessary to enable the merge with the geojson data, that only has names, not codes for identifying the regions. Also, the redundant "- Natal - RN" must be removed too.

In [10]:
new_columns = {'V_x':'income', 'D3C':'id', 'D3N':'name', 'V_y':'inhabitants'}
neighborhood_df.rename(columns=new_columns, inplace=True)

# normalize the names
neighborhood_df['name'] = neighborhood_df.name.str.split(pat=' - ', n=1, expand=True)[0]
neighborhood_df['name_norm'] = neighborhood_df.name.apply(lambda x:unidecode.unidecode(x))

In [11]:
neighborhood_df.head()

,id,name,income,inhabitants,name_norm
0,2408102001,Santos Reis,984.31,2989,Santos Reis
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio
2,2408102003,Rocas,969.39,5806,Rocas
3,2408102004,Ribeira,2825.82,1453,Ribeira
4,2408102005,Petrópolis,4736.63,3288,Petropolis


Last but not least, we will add the administrative regions or zones, as they are known, to which the districts belong to the Ordinary Law nº 03878/89 of the municipality of Natal. This information was previously inserted into a JSON file

In [12]:
with open('../data/interim/zones.json') as fp:
    zones = json.load(fp)
neighborhood_df['zone'] = neighborhood_df['name'].map(zones) 

In [13]:
neighborhood_df.head()

,id,name,income,inhabitants,name_norm,zone
0,2408102001,Santos Reis,984.31,2989,Santos Reis,East
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio,East
2,2408102003,Rocas,969.39,5806,Rocas,East
3,2408102004,Ribeira,2825.82,1453,Ribeira,East
4,2408102005,Petrópolis,4736.63,3288,Petropolis,East


### Retrieving the centroid of each neighborhood
The GeoJSON file provides a polygon that describes Natal geographically. If we get its central point, which means its centroid, we can measure the distance between the center of the area and a point of interest as school, bank, police station, etc. [geopandas](http://geopandas.org/) module is responsible for dealing with this data.

In [14]:
# load the GeoJSON data and use 'UTF-8'encoding
geojson_natal_file = requests.get('https://github.com/nymarya/data-science-one/blob/master/Lesson%2314/natal.geojson?raw=true')
geo_json_natal = geojson_natal_file.json()

In [15]:
# save it in a local file
with open('../data/external/natal.geojson', 'w') as file:  
    json.dump(geo_json_natal, file)
    
geojson_df = gpd.read_file("../data/external/natal.geojson")

Let's make sure that the data is all there

In [16]:
geojson_df.head()

,id,@id,admin_level,boundary,is_in,name,place,type,alt_name,wikidata,geometry
0,relation/388146,relation/388146,10,administrative,Natal,Pitimbu,suburb,boundary,None,None,"POLYGON ((-35.24373 -5.84203, -35.24379 -5.842..."
1,relation/388147,relation/388147,10,administrative,Natal,Planalto,suburb,boundary,None,None,"POLYGON ((-35.25382 -5.86629, -35.25361 -5.866..."
2,relation/397022,relation/397022,10,administrative,Natal,Ponta Negra,suburb,boundary,None,None,"POLYGON ((-35.18902 -5.89079, -35.18895 -5.890..."
3,relation/1230018,relation/1230018,10,administrative,Natal,Neópolis,suburb,boundary,None,None,"POLYGON ((-35.20031 -5.87341, -35.19996 -5.873..."
4,relation/1230020,relation/1230020,10,administrative,Natal,Capim Macio,suburb,boundary,None,None,"POLYGON ((-35.18946 -5.87242, -35.18907 -5.871..."


In [17]:
geojson_df.name.unique()

array(['Pitimbu', 'Planalto', 'Ponta Negra', 'Neópolis', 'Capim Macio',
       'Lagoa Azul', 'Pajuçara', 'Lagoa Seca', 'Barro Vermelho',
       'Candelária', 'Praia do Meio', 'Rocas', 'Santos Reis', 'Redinha',
       'Salinas', 'Igapó', 'Nossa Senhora da Apresentação', 'Potengi',
       'Ribeira', 'Cidade Alta', 'Alecrim', 'Nordeste', 'Quintas',
       'Bom Pastor', 'Dix-Sept Rosado', 'Nossa Senhora de Nazaré',
       'Lagoa Nova', 'Mãe Luiza', 'Nova Descoberta', 'Tirol',
       'Petrópolis', 'Areia Preta', 'Cidade Nova', 'Cidade da Esperança',
       'Felipe Camarão', 'Guarapes'], dtype=object)

In [18]:
n_neighborhoods1 = len(neighborhood_df.name.unique())
n_neighborhoods2 = len(geojson_df.name.unique())
assert n_neighborhoods1 == n_neighborhoods2

Knowing that the polygon is a complex structure/data type, merging the datasets using pandas is also complex. So in this step, we merge the datasets and then extract the x and y.

In [19]:
# fix typo
typo_df = geojson_df.query('name == "Felipe Camarão"')
geojson_df.loc[typo_df.index.values, 'name'] = 'Filipe Camarão'

# save each coordinate
geojson_df['name_norm'] = geojson_df.name.apply(lambda x:unidecode.unidecode(x))
neighborhood_df = neighborhood_df.merge(geojson_df[['name_norm', 'geometry']], on='name_norm')


neighborhood_df.head()

,id,name,income,inhabitants,name_norm,zone,geometry
0,2408102001,Santos Reis,984.31,2989,Santos Reis,East,"POLYGON ((-35.20819 -5.76785, -35.20420 -5.768..."
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio,East,"POLYGON ((-35.19504 -5.77323, -35.19513 -5.773..."
2,2408102003,Rocas,969.39,5806,Rocas,East,"POLYGON ((-35.19664 -5.77523, -35.19653 -5.774..."
3,2408102004,Ribeira,2825.82,1453,Ribeira,East,"POLYGON ((-35.20412 -5.76860, -35.20420 -5.768..."
4,2408102005,Petrópolis,4736.63,3288,Petropolis,East,"POLYGON ((-35.19536 -5.78434, -35.19532 -5.784..."


In [20]:
# create columns in the main df for the centroid
neighborhood_df['x'] = neighborhood_df.geometry.apply(lambda x: Polygon(x).centroid.x) # lon
neighborhood_df['y'] = neighborhood_df.geometry.apply(lambda x: Polygon(x).centroid.y) # lat 

neighborhood_df.head()

,id,name,income,inhabitants,name_norm,zone,geometry,x,y
0,2408102001,Santos Reis,984.31,2989,Santos Reis,East,"POLYGON ((-35.20819 -5.76785, -35.20420 -5.768...",-35.199888,-5.762731
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio,East,"POLYGON ((-35.19504 -5.77323, -35.19513 -5.773...",-35.195083,-5.778860
2,2408102003,Rocas,969.39,5806,Rocas,East,"POLYGON ((-35.19664 -5.77523, -35.19653 -5.774...",-35.200216,-5.774142
3,2408102004,Ribeira,2825.82,1453,Ribeira,East,"POLYGON ((-35.20412 -5.76860, -35.20420 -5.768...",-35.205768,-5.776236
4,2408102005,Petrópolis,4736.63,3288,Petropolis,East,"POLYGON ((-35.19536 -5.78434, -35.19532 -5.784...",-35.198393,-5.784591


### Calculating the distance
After a search for the bbox coordinates in [https://boundingbox.klokantech.com](https://boundingbox.klokantech.com), we create a [Pandana](https://udst.github.io/pandana/network.html) network for Natal. Pandana provides efficent ways to calculate distances.

In [21]:
bbox=[-5.90021100,-35.29126300,-5.70272200,-35.1531030]

In [22]:
net = osm.pdna_network_from_bbox(*bbox)

net.precompute(5000)
net.edges_df.head(2)

Requesting network data within bounding box from Overpass API in 1 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](-5.90021100,-35.29126300,-5.70272200,-35.15310300);>;);out;'}"
Downloaded 12,474.0KB from www.overpass-api.de in 4.41 seconds
Downloaded OSM network data within bounding box from Overpass API in 1 request(s) and 4.69 seconds
Returning OSM data with 84,321 nodes and 21,376 ways...
Edge node pairs completed. Took 34.85 seconds
Returning processed graph with 30,850 nodes and 45,707 edges...
Completed OSM data download and Pandana node and edge table creation in 41.45 seconds


,,from,to,distance
243207469,7051202996,243207469,7051202996,6.496116
243207479,7051202993,243207479,7051202993,7.418791


A quick test: retrieving the centroid of a neighborhood, and using `get_node_ids(x, y)` to get the node in the network closer to the point. That way, we can check the osmid and see if the node is really in the center of the area.

In [23]:
nordeste = neighborhood_df.query("name == 'Nordeste'")
x = nordeste.x.tolist()[0]
y = nordeste.y.tolist()[0]
neighborhood_node_id = net.get_node_ids([x], [y])
neighborhood_node_id

0    7609541658
Name: node_id, dtype: int64

The result is available [here](https://overpass-turbo.eu/s/190G).

#### Testing pandana `nearest_pois`

Let's make a quick test: search amenities for *one* neighborhood and see if we can calculate the distances. Code mostly from [Pandana demo](https://github.com/UDST/pandana/blob/dev/examples/Pandana-demo.ipynb)

In [24]:
schools = osm.node_query(*bbox, tags='"amenity"="school"')
schools.head(2)

,lat,lon,amenity,name,short_name,phone,alt_name,addr:street,name:pt,addr:city,addr:suburb
id,,,,,,,,,,,
501170977,-5.869024,-35.234268,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501170997,-5.823610,-35.222611,school,Centro de Atenção Integral a Criança e ao Adol...,CAIC,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# set_pois receives longitude (x) and then latitude (y)
net.set_pois("schools", 10000, 10, schools.lon, schools.lat)


In [26]:
# get the 10 nearest schools for each node
result = net.nearest_pois(10000, "schools", num_pois=10)
result

,1,2,3,4,5,6,7,8,9,10
id,,,,,,,,,,
243207469,4420.305176,5573.122070,6333.185059,6545.099121,6629.525879,6800.579102,6976.217773,7267.329102,7431.617188,7477.771973
243207479,3494.386963,5703.607910,5874.661133,6499.040039,7259.103027,7471.017090,7902.136230,8075.201172,8089.163086,8115.049805
243207483,2795.961914,5005.183105,5176.235840,7197.464844,7376.775879,7390.737793,7416.625000,7466.516113,7785.445801,7915.402832
243207509,1038.739014,3247.959961,3419.012939,5619.553223,5633.515137,5659.401855,5709.292969,6028.223145,6158.180176,6274.568848
243207565,1528.616943,1596.166992,1840.734985,4212.328125,4226.290039,4251.793945,4301.685059,4620.998047,4750.955078,4867.344238
...,...,...,...,...,...,...,...,...,...,...
8787909245,7575.017090,7949.073242,9567.990234,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
8787909246,7534.037109,7988.738770,9527.009766,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
8787909250,7519.299805,7847.514160,9512.273438,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000


We can find the smallest distance for the point retrieved earlier, the centroid of Nordeste, and get the distances for the 10 nearest schools around it.

In [27]:
neighborhood_node_id = net.get_node_ids([x], [y])
neighborhood_node_id

0    7609541658
Name: node_id, dtype: int64

In [28]:
result.loc[neighborhood_node_id[0],:]

1     1349.593994
2     1498.220947
3     3457.821045
4     3730.551025
5     3951.217041
6     3972.718018
7     4656.436035
8     4684.602051
9     4694.061035
10    4755.539062
Name: 7609541658, dtype: float64

#### Calcutate the distance

Now, we can repeat all these steps for all neighborhoods and save them

In [29]:
amenities = ['hospital', 'restaurant', 'bank','school', 'police', 'pharmacy']
# create string for query that must be in form: "amenity"~"hospital|bank..."
amenities_tag = '|'.join(amenities)
print(amenities_tag)

# create new columns
cols = ['nearest_' + amenity for amenity in amenities]
neighborhood_df = neighborhood_df.reindex(columns=neighborhood_df.columns.tolist() + cols)

hospital|restaurant|bank|school|police|pharmacy


In [30]:
all_pois = osm.node_query(*bbox, tags=f'"amenity"~"{amenities_tag}"')
for amenity in amenities:
    pois = all_pois.query(f"amenity == '{amenity}'")
    # set pois for amenity
    net.set_pois(amenity, 10000, 10, pois.lon, pois.lat)
    
    # get 10 nearest pois
    result = net.nearest_pois(10000, amenity, num_pois=10)
    
    
    for name in neighborhood_df.name.values:
        # get centroid
        neighborhood = neighborhood_df.query(f"name == '{name}'")
        x = neighborhood.x.tolist()[0]
        y = neighborhood.y.tolist()[0]
        
        # get node for centroid
        neighborhood_node_id = net.get_node_ids([x], [y])
        
        # get distance between closer amenity and centroid 
        distance = result.loc[neighborhood_node_id[0], 1]
        # update dataframe
        neighborhood_df.loc[neighborhood.index, f'nearest_{amenity}'] = distance

In [31]:
neighborhood_df.head()

,id,name,income,inhabitants,name_norm,zone,geometry,x,y,nearest_hospital,nearest_restaurant,nearest_bank,nearest_school,nearest_police,nearest_pharmacy
0,2408102001,Santos Reis,984.31,2989,Santos Reis,East,"POLYGON ((-35.20819 -5.76785, -35.20420 -5.768...",-35.199888,-5.762731,1258.056030,1870.130005,2876.311035,1929.688965,1418.093018,2089.927002
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio,East,"POLYGON ((-35.19504 -5.77323, -35.19513 -5.773...",-35.195083,-5.778860,1023.383972,745.338989,1912.135010,515.601990,404.269012,1105.186035
2,2408102003,Rocas,969.39,5806,Rocas,East,"POLYGON ((-35.19664 -5.77523, -35.19653 -5.774...",-35.200216,-5.774142,506.315002,644.546021,1608.114990,361.954987,473.287994,678.471985
3,2408102004,Ribeira,2825.82,1453,Ribeira,East,"POLYGON ((-35.20412 -5.76860, -35.20420 -5.768...",-35.205768,-5.776236,456.994995,197.643997,1023.872009,1043.295044,556.598022,428.792999
4,2408102005,Petrópolis,4736.63,3288,Petropolis,East,"POLYGON ((-35.19536 -5.78434, -35.19532 -5.784...",-35.198393,-5.784591,364.924011,225.037003,1040.583008,722.742004,1254.473999,441.210999


Finally, save dataset to use it in the the next step: creating maps.

In [32]:
neighborhood_df.to_csv('../data/processed/neighbourhoods.csv')